In [6]:
import pandas as pd
import numpy as np
import torch
import pickle
from tqdm import tqdm_notebook as tqdm
import os
import re
import pickle

In [7]:
torch.cuda.is_available()

True

In [8]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("gpu num: ", n_gpu)

gpu num:  1


## Load pre-trained translation model from fairseq.
You can always use another middle language or other models.

In [9]:
# Load translation model
en2ru = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-ru.single_model', tokenizer='moses', bpe='fastbpe')
ru2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.ru-en.single_model', tokenizer='moses', bpe='fastbpe')

Using cache found in /home/geekdog/.cache/torch/hub/pytorch_fairseq_main
INFO:fairseq.tasks.text_to_speech:Please install tensorboardX: pip install tensorboardX
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dl.fbaipublicfiles.com:443
DEBUG:urllib3.connectionpool:https://dl.fbaipublicfiles.com:443 "HEAD /fairseq/models/wmt19.en-ru.single_model.tar.gz HTTP/1.1" 200 0
INFO:fairseq.file_utils:loading archive file https://dl.fbaipublicfiles.com/fairseq/models/wmt19.en-ru.single_model.tar.gz from cache at /home/geekdog/.cache/torch/pytorch_fairseq/57b24ff1a88d5f14807352c9d8e17949d46ae3d0a06c1be3667929101011cdc0.8699fa0753ba14331352cc26ceb7aa46fba856bad52dd40fc97d3834418bf6f1
/home/geekdog/.cache/torch/hub/pytorch_fairseq_main/fairseq/checkpoint_utils.py:340: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbi

In [10]:
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de.single_model', tokenizer='moses', bpe='fastbpe')
de2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en.single_model', tokenizer='moses', bpe='fastbpe')

Using cache found in /home/geekdog/.cache/torch/hub/pytorch_fairseq_main
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dl.fbaipublicfiles.com:443
DEBUG:urllib3.connectionpool:https://dl.fbaipublicfiles.com:443 "HEAD /fairseq/models/wmt19.en-de.joined-dict.single_model.tar.gz HTTP/1.1" 200 0
INFO:fairseq.file_utils:loading archive file https://dl.fbaipublicfiles.com/fairseq/models/wmt19.en-de.joined-dict.single_model.tar.gz from cache at /home/geekdog/.cache/torch/pytorch_fairseq/81a0be5cbbf1c106320ef94681844d4594031c94c16b0475be11faa5a5120c48.63b093d59e7e0814ff799bb965ed4cbde30200b8c93a44bf8c1e5e98f5c54db3
DEBUG:hydra.core.utils:Setting JobRuntime:name=utils
DEBUG:hydra.core.utils:Setting JobRuntime:name=utils
INFO:fairseq.tasks.translation:[en] dictionary: 42024 types
INFO:fairseq.tasks.translation:[de] dictionary: 42024 types
INFO:fairseq.models.fairseq_model:{'_name': None, 'common': {'_name': None, 'no_progress_bar': True, 'log_interval': 100, 'log_format': 'simpl

In [11]:
en2ru.cuda()
ru2en.cuda()

GeneratorHubInterface(
  (models): ModuleList(
    (0): TransformerModel(
      (encoder): TransformerEncoderBase(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(31232, 1024, padding_idx=1)
        (embed_positions): SinusoidalPositionalEmbedding()
        (layers): ModuleList(
          (0-5): 6 x TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dropout_module): FairseqDropout()
            (activation_dropout_module): FairseqDropout()
            (fc1): 

In [12]:
en2de.cuda()
de2en.cuda()

GeneratorHubInterface(
  (models): ModuleList(
    (0): TransformerModel(
      (encoder): TransformerEncoderBase(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(42024, 1024, padding_idx=1)
        (embed_positions): SinusoidalPositionalEmbedding()
        (layers): ModuleList(
          (0-5): 6 x TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dropout_module): FairseqDropout()
            (activation_dropout_module): FairseqDropout()
            (fc1): 

## Load data

In [13]:
path = './'
train_df = pd.read_csv(path+'train.csv', header=None)
train_df.head()

,0,1,2
0,1,E. D. Abbott Ltd,Abbott of Farnham E D Abbott Limited was a Br...
1,1,Schwan-Stabilo,Schwan-STABILO is a German maker of pens for ...
2,1,Q-workshop,Q-workshop is a Polish company located in Poz...
3,1,Marvell Software Solutions Israel,Marvell Software Solutions Israel known as RA...
4,1,Bergan Mercy Medical Center,Bergan Mercy Medical Center is a hospital loc...


In [14]:
train_labels = [v-1 for v in train_df[0]]
train_text = [v for v in train_df[2]]

In [15]:
train_text[0]

' Abbott of Farnham E D Abbott Limited was a British coachbuilding business based in Farnham Surrey trading under that name from 1929. A major part of their output was under sub-contract to motor vehicle manufacturers. Their business closed in 1972.'

In [16]:
len(train_text)

560000

In [17]:
max(train_labels)

13

In [18]:
# split and get our unlabeled training data
def train_val_split(labels, n_labeled_per_class, n_labels, seed = 0):
    np.random.seed(seed)
    labels = np.array(labels)
    train_labeled_idxs = []
    train_unlabeled_idxs = []
    val_idxs = []

    for i in range(n_labels):
        idxs = np.where(labels == i)[0]
        np.random.shuffle(idxs)
        train_labeled_idxs.extend(idxs[:n_labeled_per_class])
        train_unlabeled_idxs.extend(idxs[n_labeled_per_class : n_labeled_per_class + 10000])
        val_idxs.extend(idxs[-3000:])
    
    np.random.shuffle(train_labeled_idxs)
    np.random.shuffle(train_unlabeled_idxs)
    np.random.shuffle(val_idxs)
    return train_labeled_idxs, train_unlabeled_idxs, val_idxs

In [19]:
train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(train_labels, 500, 14)

In [20]:
len(train_unlabeled_idxs)

140000

In [23]:
idxs = train_unlabeled_idxs

In [24]:
idxs[0]

44026

## Back translation process
You can tune the temperature in the translation process to control the diversity.

In [25]:
# back translate using Russian as middle language
def translate_ru(start, end, file_name):
    trans_result = {}
    for id in tqdm(range(start, end)):
        trans_result[idxs[id]] = ru2en.translate(en2ru.translate(train_text[idxs[id]],  sampling = True, temperature = 0.9),  sampling = True, temperature = 0.9)
        if id % 500 == 0:
            with open(file_name, 'wb') as f:
                pickle.dump(trans_result, f)
    with open(file_name, 'wb') as f:
        pickle.dump(trans_result, f)

In [26]:
# back translate using German as middle language
def translate_de(start, end, file_name):
    trans_result = {}
    for id in tqdm(range(start, end)):
        trans_result[idxs[id]] = de2en.translate(en2de.translate(train_text[idxs[id]],  sampling = True, temperature = 0.9),  sampling = True, temperature = 0.9)
        if id % 500 == 0:
            with open(file_name, 'wb') as f:
                pickle.dump(trans_result, f)
    with open(file_name, 'wb') as f:
        pickle.dump(trans_result, f)

In [35]:
# translate_de(0, 140000, 'de_1.pkl') # fail at 17175? (0 -> 17000)
# translate_de(17501,140000, 'de_1.pkl') #fail at 61054 (17501 -> 78500)
# translate_de(79001, 140000, 'de_1.pkl') # fail at 44756 (79001 -> 123500)
# translate_de(124001, 140000, 'de_1.pkl')
translate_de(17000, 17500, 'de_1.pkl')

/tmp/ipykernel_165004/3549724842.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for id in tqdm(range(start, end)):


  0%|          | 0/500 [00:00<?, ?it/s]

INFO:fairseq.tasks.fairseq_task:can_reuse_epoch_itr = False
INFO:fairseq.tasks.fairseq_task:reuse_dataloader = True
INFO:fairseq.tasks.fairseq_task:rebuild_batches = False
INFO:fairseq.tasks.fairseq_task:creating new batches for epoch 1
INFO:fairseq.tasks.fairseq_task:can_reuse_epoch_itr = False
INFO:fairseq.tasks.fairseq_task:reuse_dataloader = True
INFO:fairseq.tasks.fairseq_task:rebuild_batches = False
INFO:fairseq.tasks.fairseq_task:creating new batches for epoch 1
INFO:fairseq.tasks.fairseq_task:can_reuse_epoch_itr = False
INFO:fairseq.tasks.fairseq_task:reuse_dataloader = True
INFO:fairseq.tasks.fairseq_task:rebuild_batches = False
INFO:fairseq.tasks.fairseq_task:creating new batches for epoch 1
INFO:fairseq.tasks.fairseq_task:can_reuse_epoch_itr = False
INFO:fairseq.tasks.fairseq_task:reuse_dataloader = True
INFO:fairseq.tasks.fairseq_task:rebuild_batches = False
INFO:fairseq.tasks.fairseq_task:creating new batches for epoch 1
INFO:fairseq.tasks.fairseq_task:can_reuse_epoch_itr 

Exception: Size of sample #0 is invalid (=(2878, 0)) since max_positions=(1024, 1024), skip this example with --skip-invalid-size-inputs-valid-test

In [31]:
import pickle
# load pickle file 
with open('de_3.pkl', 'rb') as f:
    train_bt = pickle.load(f)

In [ ]:
Exception: Size of sample #0 is invalid (=(1745, 0)) since max_positions=(1024, 1024), skip this example with --skip-invalid-size-inputs-valid-test"

In [42]:
# Helper to truncate if tokens exceed limit
def safe_translate(text, model, max_len=1024, sampling=True, temperature=0.9):
    tokens = model.encode(text)
    if len(tokens) > max_len:
        tokens = tokens[:max_len]
        text = model.decode(tokens)
    return model.translate(text, sampling=sampling, temperature=temperature)




In [43]:
import textwrap

text = train_text[idxs[17175]]  # your full Coraebus genus text
chunks = textwrap.wrap(text, width=1000, break_long_words=False)

translated_chunks = []
for chunk in chunks:
    de_text = safe_translate(chunk, en2de)
    back_text = safe_translate(de_text, de2en)
    translated_chunks.append(back_text)

final_text = ' '.join(translated_chunks)


INFO:fairseq.tasks.fairseq_task:can_reuse_epoch_itr = False
INFO:fairseq.tasks.fairseq_task:reuse_dataloader = True
INFO:fairseq.tasks.fairseq_task:rebuild_batches = False
INFO:fairseq.tasks.fairseq_task:creating new batches for epoch 1
INFO:fairseq.tasks.fairseq_task:can_reuse_epoch_itr = False
INFO:fairseq.tasks.fairseq_task:reuse_dataloader = True
INFO:fairseq.tasks.fairseq_task:rebuild_batches = False
INFO:fairseq.tasks.fairseq_task:creating new batches for epoch 1
INFO:fairseq.tasks.fairseq_task:can_reuse_epoch_itr = False
INFO:fairseq.tasks.fairseq_task:reuse_dataloader = True
INFO:fairseq.tasks.fairseq_task:rebuild_batches = False
INFO:fairseq.tasks.fairseq_task:creating new batches for epoch 1
INFO:fairseq.tasks.fairseq_task:can_reuse_epoch_itr = False
INFO:fairseq.tasks.fairseq_task:reuse_dataloader = True
INFO:fairseq.tasks.fairseq_task:rebuild_batches = False
INFO:fairseq.tasks.fairseq_task:creating new batches for epoch 1
INFO:fairseq.tasks.fairseq_task:can_reuse_epoch_itr 

In [41]:
print(train_text[idxs[17175]])
token = en2de.encode(train_text[idxs[17175]])

 Coraebus is a genus of beetles in the family Buprestidae containing the following species: Coraebus aculeatus Ganglbauer 1890 Coraebus acutus Thomson 1879 Coraebus aeneopictus (Kerremans 1895) Coraebus aequalipennis Fairmaire 1888 Coraebus aesopus Kerremans 1913 Coraebus akiyamai Kubán 1997 Coraebus albivestitus Bellamy 1991 Coraebus aldo Kubán 1996 Coraebus amabilis Kerremans 1895 Coraebus amamianus Kurosawa 1985 Coraebus amplithorax (Fairmaire 1889) Coraebus andreorum Kubán 1996 Coraebus andrewesi Obenberger 1922 Coraebus annamensis Descarpentries & Villiers 1967 Coraebus anniae Obenberger 1928 Coraebus auberti Théry 1923 Coraebus auriventris Kerremans 1912 Coraebus aurofasciatus (Hope 1831) Coraebus baoshanensis Kubán 1996 Coraebus battareli Descarpentries & Villiers 1967 Coraebus baylei Bourgoin 1922 Coraebus becvari Kubán 1995 Coraebus bilyi Kubán 1996 Coraebus bilyianus Kubán 1995 Coraebus binhensis Descarpentries & Villiers 1967 Coraebus bivestitus Bellamy 1991 Coraebus blandus

In [46]:
# Helper to truncate if tokens exceed limit
def safe_translate(text, model, max_len=1024, sampling=True, temperature=0.9):
    tokens = model.encode(text)
    if len(tokens) > max_len:
        tokens = tokens[:max_len]
        text = model.decode(tokens)
    return model.translate(text, sampling=sampling, temperature=temperature)

def translate_by_chunk(text):

    chunks = textwrap.wrap(text, width=1000, break_long_words=False)

    translated_chunks = []
    for chunk in chunks:
        de_text = safe_translate(chunk, en2de)
        back_text = safe_translate(de_text, de2en)
        translated_chunks.append(back_text)

    final_text = ' '.join(translated_chunks)
    return final_text


# back translate using German as middle language
def safe_translate_de(start, end, file_name):
    trans_result = {}
    for id in tqdm(range(start, end)):
        try:
            trans_result[idxs[id]] = de2en.translate(en2de.translate(train_text[idxs[id]],  sampling = True, temperature = 0.9),  sampling = True, temperature = 0.9)
        except:
            trans_result[idxs[id]] = translate_by_chunk(train_text[idxs[id]])
            
        if id % 500 == 0:
            with open(file_name, 'wb') as f:
                pickle.dump(trans_result, f)
    with open(file_name, 'wb') as f:
        pickle.dump(trans_result, f)

In [47]:
safe_translate_de(78501, 79000, 'de_1.pkl')

/tmp/ipykernel_165004/2295713809.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for id in tqdm(range(start, end)):


  0%|          | 0/499 [00:00<?, ?it/s]

INFO:fairseq.tasks.fairseq_task:can_reuse_epoch_itr = False
INFO:fairseq.tasks.fairseq_task:reuse_dataloader = True
INFO:fairseq.tasks.fairseq_task:rebuild_batches = False
INFO:fairseq.tasks.fairseq_task:creating new batches for epoch 1
INFO:fairseq.tasks.fairseq_task:can_reuse_epoch_itr = False
INFO:fairseq.tasks.fairseq_task:reuse_dataloader = True
INFO:fairseq.tasks.fairseq_task:rebuild_batches = False
INFO:fairseq.tasks.fairseq_task:creating new batches for epoch 1
INFO:fairseq.tasks.fairseq_task:can_reuse_epoch_itr = False
INFO:fairseq.tasks.fairseq_task:reuse_dataloader = True
INFO:fairseq.tasks.fairseq_task:rebuild_batches = False
INFO:fairseq.tasks.fairseq_task:creating new batches for epoch 1
INFO:fairseq.tasks.fairseq_task:can_reuse_epoch_itr = False
INFO:fairseq.tasks.fairseq_task:reuse_dataloader = True
INFO:fairseq.tasks.fairseq_task:rebuild_batches = False
INFO:fairseq.tasks.fairseq_task:creating new batches for epoch 1
INFO:fairseq.tasks.fairseq_task:can_reuse_epoch_itr 